اشهر اربع مكتبات في spark
---------------------------------

1. spark sql :- data stucture by sql لمعالجه البيانات المهيكله     
2. spark streaming :-  to provess the flow of the data in the real time streaming
3. MLlib :- to machine learning that built in spark 
4. GraphX :-  to process and anlysis the data in graph form 

In [48]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate()

sc= spark.sparkContext
#RDD
dataRDD= sc.parallelize([("Brooke",20),("Denny",31),("Jules",30),("TD",35),("Brooke",25),("Jules",30),("Denny",51),("TD",15)])

dataRDD.collect()

[('Brooke', 20),
 ('Denny', 31),
 ('Jules', 30),
 ('TD', 35),
 ('Brooke', 25),
 ('Jules', 30),
 ('Denny', 51),
 ('TD', 15)]

In [2]:
data_frame= spark.createDataFrame([("Brooke",20),("Denny",31),("Jules",30),("TD",35),
("Brooke",25),("Jules",30),("Denny",51),("TD",15)],["name","age"])

#data_frame.printSchema()
#data_frame.show()  
data_frame.show(5)  
r=data_frame.collect()[0]
r.name

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
+------+---+
only showing top 5 rows


'Brooke'

In [5]:
# استخدام group by عشان اعمل متوسط عن طريق العمر 
 
avg_age=data_frame.groupBy('name').avg('age')
avg_age.show()

# عشان اعرف عدد البرتشننز الي هو 1 بس 
avg_age.rdd.getNumPartitions() # فوق كان في 2 بارتشن بس هنا 1 بس لانه عمل جروب باي يعني عمل شفل بين الي فوق عشان يطلع ببرتشن واحد بس 

avg_age.head(2)


+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Jules|    30.0|
|    TD|    25.0|
| Denny|    41.0|
+------+--------+



[Row(name='Brooke', avg(age)=22.5), Row(name='Jules', avg(age)=30.0)]

agg function 
------------

In [ ]:
#يعني اي agg 
data = [
    ("Ahmed", "IT", 3000),
    ("Sara", "IT", 4000),
    ("Ali", "HR", 2000),
    ("Mona", "HR", 2500)
]

columns = ["name", "department", "salary"]

df = spark.createDataFrame(data, columns)
df.show()


# تجميع بدون group by
from pyspark.sql import functions as F

df.agg(
    F.sum("salary").alias("total_salary"),
    F.avg("salary").alias("avg_salary")
).show()


# تجميع مع group by 
df.groupBy("department").agg(
    F.sum("salary").alias("total_salary"),
    F.avg("salary").alias("avg_salary")
).show()



+-----+----------+------+
| name|department|salary|
+-----+----------+------+
|Ahmed|        IT|  3000|
| Sara|        IT|  4000|
|  Ali|        HR|  2000|
| Mona|        HR|  2500|
+-----+----------+------+



+------------+----------+
|total_salary|avg_salary|
+------------+----------+
|       11500|    2875.0|
+------------+----------+



+----------+------------+----------+
|department|total_salary|avg_salary|
+----------+------------+----------+
|        IT|        7000|    3500.0|
|        HR|        4500|    2250.0|
+----------+------------+----------+



git data from json file 
--------------------------

In [ ]:
# import col 
from pyspark.sql.functions import col 


df=spark.read.json('/home/ahmed-refat/Downloads/people.json')
name=df['name']

# استخدام sql statments
df.select('name')
df_age =df.select('age')

# where condition
df_age.where(col('age')>20).show()


+---+
|age|
+---+
| 34|
| 22|
+---+



==================================================================================================================

تجربه التعامل مع داتا فريم كامله 
------------------------------

In [49]:
from pyspark.sql.functions import *
import pyspark.sql.functions as fn # دي احسن من الي فوق عشان ميبقاش في مضاربه مع البايثون نفسهالا

In [52]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn

spark = SparkSession.builder.appName("HandsOnDF").getOrCreate()

data = [
    (1, "Ahmed", 25000.0, "sales", 2),
    (2, "Mona", 32000.0, "sales", 5),
    (3, "Omar", 28000.0, "engineering", 3),
    (4, "Sara", 40000.0, "engineering", 7),
    (5, "Hany", 15000.0, "hr", 1),
    (6, "Nora", 22000.0, "hr", 4)
]

cols = ["id", "name", "salary", "department", "years"]

df = spark.createDataFrame(data, cols,)
df.show(truncate=False) # اعرض كل الداتا من غير متقص حاجه لان غالبا show بتديني اول 20 بس 
df.printSchema()


+---+-----+-------+-----------+-----+
|id |name |salary |department |years|
+---+-----+-------+-----------+-----+
|1  |Ahmed|25000.0|sales      |2    |
|2  |Mona |32000.0|sales      |5    |
|3  |Omar |28000.0|engineering|3    |
|4  |Sara |40000.0|engineering|7    |
|5  |Hany |15000.0|hr         |1    |
|6  |Nora |22000.0|hr         |4    |
+---+-----+-------+-----------+-----+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- department: string (nullable = true)
 |-- years: long (nullable = true)



In [53]:
# Step 1: عمليات حسابية وwithColumn
# 1. bonus = 10% of salary
df = df.withColumn("bonus", fn.col("salary") * 0.10)

# 2. total_comp = salary + bonus
df = df.withColumn("total_comp", fn.col("salary") + fn.col("bonus"))

# 3. مثال تعديل: زيادة salary بنسبة 5% ثم نحفظه في عمود جديد
df = df.withColumn("salary_plus_5pct", fn.col("salary") * 1.05)

df.show()


+---+-----+-------+-----------+-----+------+----------+----------------+
| id| name| salary| department|years| bonus|total_comp|salary_plus_5pct|
+---+-----+-------+-----------+-----+------+----------+----------------+
|  1|Ahmed|25000.0|      sales|    2|2500.0|   27500.0|         26250.0|
|  2| Mona|32000.0|      sales|    5|3200.0|   35200.0|         33600.0|
|  3| Omar|28000.0|engineering|    3|2800.0|   30800.0|         29400.0|
|  4| Sara|40000.0|engineering|    7|4000.0|   44000.0|         42000.0|
|  5| Hany|15000.0|         hr|    1|1500.0|   16500.0|         15750.0|
|  6| Nora|22000.0|         hr|    4|2200.0|   24200.0|         23100.0|
+---+-----+-------+-----------+-----+------+----------+----------------+



In [54]:
# اضافه عمود ثابت القيمه 
df= df.withColumn('county',fn.lit('egypt'))

# اضافه عمود جديد فيها جنيور واو سنيور علي حسب هو اكتر من 5 سنين خيره او لا 

df=df.withColumn('level',
                 fn.when(fn.col('years')>= 5,'senior').otherwise('junior'))


# use and & , or |

df = df.withColumn('status',
                   fn.when((fn.col('salary')>= 30000)  & (fn.col('department')== 'sales'),'top sales' ).otherwise('normal')

)

df.show()

+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+
| id| name| salary| department|years| bonus|total_comp|salary_plus_5pct|county| level|   status|
+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+
|  1|Ahmed|25000.0|      sales|    2|2500.0|   27500.0|         26250.0| egypt|junior|   normal|
|  2| Mona|32000.0|      sales|    5|3200.0|   35200.0|         33600.0| egypt|senior|top sales|
|  3| Omar|28000.0|engineering|    3|2800.0|   30800.0|         29400.0| egypt|junior|   normal|
|  4| Sara|40000.0|engineering|    7|4000.0|   44000.0|         42000.0| egypt|senior|   normal|
|  5| Hany|15000.0|         hr|    1|1500.0|   16500.0|         15750.0| egypt|junior|   normal|
|  6| Nora|22000.0|         hr|    4|2200.0|   24200.0|         23100.0| egypt|junior|   normal|
+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+



In [55]:
# تحويل الاسم كله لحروف كبيرة
df = df.withColumn("name_upper", fn.upper(fn.col("name")))

# تحويل القسم كله لحروف صغيرة
df = df.withColumn("department_lower", fn.lower(fn.col("department")))

# الاسم الأول بحرف كبير فقط
df = df.withColumn("name_initcap", fn.initcap(fn.col("name")))

df.show(truncate=False)


+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+
|id |name |salary |department |years|bonus |total_comp|salary_plus_5pct|county|level |status   |name_upper|department_lower|name_initcap|
+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+
|1  |Ahmed|25000.0|sales      |2    |2500.0|27500.0   |26250.0         |egypt |junior|normal   |AHMED     |sales           |Ahmed       |
|2  |Mona |32000.0|sales      |5    |3200.0|35200.0   |33600.0         |egypt |senior|top sales|MONA      |sales           |Mona        |
|3  |Omar |28000.0|engineering|3    |2800.0|30800.0   |29400.0         |egypt |junior|normal   |OMAR      |engineering     |Omar        |
|4  |Sara |40000.0|engineering|7    |4000.0|44000.0   |42000.0         |egypt |senior|normal   |SARA      |engineering     |Sara        |
|5  |Hany |15000.0|hr         |1  

In [56]:
# دمج اعمده مباشر بدون فواصل 
import pyspark.sql.functions as fn
df= df.withColumn('display1',fn.concat(fn.col('name'),fn.col('department')))

# دمج مع فصله في المنتصف

df =df.withColumn('display',fn.concat_ws('-',fn.col('name'),fn.upper(fn.col('department'))))

df.show()

+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+---------------+----------------+
| id| name| salary| department|years| bonus|total_comp|salary_plus_5pct|county| level|   status|name_upper|department_lower|name_initcap|       display1|         display|
+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+---------------+----------------+
|  1|Ahmed|25000.0|      sales|    2|2500.0|   27500.0|         26250.0| egypt|junior|   normal|     AHMED|           sales|       Ahmed|     Ahmedsales|     Ahmed-SALES|
|  2| Mona|32000.0|      sales|    5|3200.0|   35200.0|         33600.0| egypt|senior|top sales|      MONA|           sales|        Mona|      Monasales|      Mona-SALES|
|  3| Omar|28000.0|engineering|    3|2800.0|   30800.0|         29400.0| egypt|junior|   normal|      OMAR|     engineering|        Omar|Omarengi

In [57]:
# whenهنعمل فيلتر ويعتبر هو هونفس التاثير بتاع 
df.filter(fn.col("department") == "sales").show(truncate=False)


+---+-----+-------+----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+----------+-----------+
|id |name |salary |department|years|bonus |total_comp|salary_plus_5pct|county|level |status   |name_upper|department_lower|name_initcap|display1  |display    |
+---+-----+-------+----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+----------+-----------+
|1  |Ahmed|25000.0|sales     |2    |2500.0|27500.0   |26250.0         |egypt |junior|normal   |AHMED     |sales           |Ahmed       |Ahmedsales|Ahmed-SALES|
|2  |Mona |32000.0|sales     |5    |3200.0|35200.0   |33600.0         |egypt |senior|top sales|MONA      |sales           |Mona        |Monasales |Mona-SALES |
+---+-----+-------+----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+----------+-----------+



In [ ]:
# select
df.select(fn.col("name").alias("employee_name"), "total_comp").show(truncate=False)
# alias بتغير اسم العمود لاي حاجه انا هكنبها 

+-------------+----------+
|employee_name|total_comp|
+-------------+----------+
|Ahmed        |27500.0   |
|Mona         |35200.0   |
|Omar         |30800.0   |
|Sara         |44000.0   |
|Hany         |16500.0   |
|Nora         |24200.0   |
+-------------+----------+



In [65]:
# ترتيب البيانات علي حسب تصاعدي او تنازلي 
df.orderBy(fn.col('department')) # ترتيب تصاعدي 
df.orderBy (df['salary'].desc()) #  ترتيب تنازلي 



# حسب department تصاعدي، و salary تنازلي
df.orderBy(df['department'],df['salary'].desc()).show()


+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+---------------+----------------+
| id| name| salary| department|years| bonus|total_comp|salary_plus_5pct|county| level|   status|name_upper|department_lower|name_initcap|       display1|         display|
+---+-----+-------+-----------+-----+------+----------+----------------+------+------+---------+----------+----------------+------------+---------------+----------------+
|  4| Sara|40000.0|engineering|    7|4000.0|   44000.0|         42000.0| egypt|senior|   normal|      SARA|     engineering|        Sara|Saraengineering|Sara-ENGINEERING|
|  3| Omar|28000.0|engineering|    3|2800.0|   30800.0|         29400.0| egypt|junior|   normal|      OMAR|     engineering|        Omar|Omarengineering|Omar-ENGINEERING|
|  6| Nora|22000.0|         hr|    4|2200.0|   24200.0|         23100.0| egypt|junior|   normal|      NORA|              hr|        Nora|        

In [ ]:
# التجميع (Aggregation) باستخدام groupBy + agg

